ação.
API: https://www.sptrans.com.br/desenvolvedores
ENDPOINTS:https://www.sptrans.com.br/desenvolvedores/api-do-olho-vivo-guia-de-r
eferencia/documentacao-api/
Utilizar o GTFS da SPTRANS para dados complementares (Dados
Estáticos/Cadastrais).
Enriquecer os dados de paradas com o endereço da localidade, através de latitude e
longitude.
Dados do GTFS: https://gtfs.org/documentation/schedule/reference/


In [1]:
import requests
import pandas as pd
API_TOKEN = "36b9de7cf0f0bcd8cdde826217615c1b5d2e7b71c80397ec50d1ec7620b4f872"
BASE_URL = "http://api.olhovivo.sptrans.com.br/v2.1"
BASE_PATH = '.'


In [2]:
session = requests.Session()
def authenticate(api_token):
     # Define the authentication endpoint
    auth_endpoint = f"{BASE_URL}/Login/Autenticar?token={api_token}"
    
    try:
        # Make a POST request to authenticate
        response = session.post(auth_endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return response
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False


result = authenticate(API_TOKEN)
print(result.text)

true


In [3]:
def buscar_linha_sentido(session, termosBusca):
     # Define the authentication endpoint
    endpoint = f"{BASE_URL}/Linha/Buscar?termosBusca={termosBusca}"
    
    try:
        # Make a POST request to authenticate
        response = session.get(endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return response
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False


result = buscar_linha_sentido(session, '8000')
print(result.text)



[{"cl":2506,"lc":false,"lt":"8000","sl":1,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":35274,"lc":false,"lt":"8000","sl":2,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":1273,"lc":false,"lt":"8000","sl":1,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":34041,"lc":false,"lt":"8000","sl":2,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"}]


In [4]:
import json

def generic_get(session, endpoint):
     # Define the authentication endpoint
    endpoint = BASE_URL + endpoint    
    # print(endpoint)
    try:
        # Make a POST request to authenticate
        response = session.get(endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return response
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False

def kmz(session):
    return generic_get(session, '/KMZ')

def parada_buscar(session, termos_busca):
    return generic_get(session, f"/Parada/Buscar?termosBusca='{termos_busca}'")

def linha_buscar(session, termos_busca):
    return generic_get(session, f"/Linha/Buscar?termosBusca='{termos_busca}'")

def parada_buscar_por_linha(session, codigo_linha):
    return generic_get(session, f"/Parada/BuscarParadasPorLinha?codigoLinha={codigo_linha}")

def previsao_parada(session, codigo_parada):
    return generic_get(session, f'/Previsao/Parada?codigoParada={codigo_parada}')

def previsao_linhas(session, codigo_linha):
    return generic_get(session, f'/Previsao/Linha?codigoLinha={codigo_linha}')

def previsao_parada_all(session):
    return generic_get(session, f'/Previsao')
    
def posicao(session):
    return generic_get(session, f'/Posicao')

def posicao_por_linha(session, codigo_linha):
    return generic_get(session, f"/Posicao/Linha?codigoLinha={codigo_linha}")

# paradas = parada_buscar(session, 'rebouças');
# print(json.dumps(result.json(), indent=4))

# paradas = parada_buscar(session, 'rebouças');
# print(json.dumps(result.json(), indent=4))

# result = parada_buscar_por_linha(session, '1171')
# print(json.dumps(result.json(), indent=4))

# result = previsao_parada(session, '630015011')
# print(json.dumps(result.json(), indent=2))

# result = posicao(session)
# print(json.dumps(result.json(), indent=2))

# result = posicao_por_linha(session, 1171)
# print(json.dumps(result.json(), indent=2))

# result = previsao_parada_all(session)
# print(json.dumps(result.json(), indent=2))

result = previsao_linhas(session, '1171')
# print(json.dumps(result.json(), indent=2))


# paradas = linha_buscar(session, 'USP');
# print(json.dumps(result.json(), indent=4))


In [5]:
df_trips = pd.read_csv(f'{BASE_PATH}/trips.txt', header=0)
df_trips.head()
print(df_trips.shape)
trips = df_trips[['trip_headsign']].values
trips = set([x[0].replace('-','') for x in trips])
print(len(trips))
list(trips)[:10]

(2241, 6)
674


['Jd. São Paulo',
 'Vl. Ayrosa',
 'Metrô Alto Do Ipiranga',
 'Jd. São Nicolau',
 'Vl. Jacui',
 'Jd. Herplin',
 'Jd. Belém',
 'Metrô Vl. Matilde',
 'Jd. Ester',
 'Lgo. Do Paissandú']

In [6]:
# import time
# import json

# with open('linha-codigo.json', 'a', encoding='utf-8', errors='replace') as output_file:
#     for i, trip in enumerate(trips): 
#         print(i)
#         try:
#             result = linha_buscar(session, trip)
#             if result.status_code != 200:
#                 print(f'Trip {trip} devolveu {result.status_code}: {result.text}')
#                 continue
#             result = result.json()
#             output_file.write(json.dumps(result))
#             output_file.write('\n')

#         except Exception as e:
#             print(e)

#     output_file.flush()
    

In [7]:
!head -n 2 linha-codigo.json

[]
[{"cl": 775, "lc": false, "lt": "7013", "sl": 1, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "PQ. ARARIBA"}, {"cl": 33543, "lc": false, "lt": "7013", "sl": 2, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "PQ. ARARIBA"}, {"cl": 496, "lc": false, "lt": "7458", "sl": 1, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "JD. BOA VISTA"}, {"cl": 33264, "lc": false, "lt": "7458", "sl": 2, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "JD. BOA VISTA"}, {"cl": 2574, "lc": false, "lt": "8019", "sl": 1, "tl": 31, "tp": "TERM. VL. S\u00d4NIA", "ts": "PQ. CONTINENTAL"}, {"cl": 35342, "lc": false, "lt": "8019", "sl": 2, "tl": 31, "tp": "TERM. VL. S\u00d4NIA", "ts": "PQ. CONTINENTAL"}, {"cl": 2074, "lc": false, "lt": "8023", "sl": 1, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "CDHU MUNCK"}, {"cl": 34842, "lc": false, "lt": "8023", "sl": 2, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "CDHU MUNCK"}, {"cl": 2076, "lc": false, "lt": "8025", "sl": 1, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "

In [8]:
total = 0
cl = {}

with open('linha-codigo.json', 'r', encoding='utf-8', errors='replace') as input_file:
    for row in input_file:
        linhas = json.loads(row)
        total += len(linhas)
        for l in linhas:
            cl[l.get('cl')] = l 
            



In [9]:
from tqdm import tqdm
trips = []

with open('linhas_from_api.json', 'w', encoding='utf-8', errors='replace') as output_file:
    for k,v in tqdm(cl.items()):
        trips.append(k)
        output_file.write(json.dumps(v))
        output_file.write('\n')



100%|██████████| 2658/2658 [00:00<00:00, 272431.94it/s]


In [10]:

# previsao_linhas(session, '198').json()

In [11]:
len(trips)

2658

In [12]:
import time
import json
from tqdm import tqdm

        
with open('previsao2.json', 'a', encoding='utf-8', errors='replace') as output_file:
    while True:
        c = 0
        for trip in tqdm(trips): 
            try:
                result = previsao_linhas(session, trip)
                if result.status_code != 200:
                    print(f'Trip {trip} devolveu {result.status_code}: {result.text}')
                    continue
                c += 1
                result = result.json()
                linhas = result.get('l',[])
                # print(f'{result["ps"]} -> Total de linhas: {len(linhas):,}')

                output_file.write(json.dumps(result))
                output_file.write('\n')

            except Exception as e:
                print(e)
            
        output_file.flush()
        print(f'sleep: {c:,} linhas')
        time.sleep(30)


    

100%|██████████| 2658/2658 [02:35<00:00, 17.11it/s]


sleep: 2,658 linhas


100%|██████████| 2658/2658 [02:41<00:00, 16.44it/s]


sleep: 2,658 linhas


KeyboardInterrupt: 